In [1]:
import psycopg2
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyecharts
import urllib3,time,csv,datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.finance as mpf
import matplotlib.dates as mpd
import plotly.offline as py_offline
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.graph_objs import *
from IPython.display import clear_output

%matplotlib inline

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "C:\Users\Brad Sun\Anaconda3\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [3]:
"""
IP：128.199.97.202
    Port：22
    username：root
    password：QZDqzd65976787

数据库：PostgreSQL
port: 5432
用户名：postgres
密码：tongKen123

"""

'\nIP：128.199.97.202\n    Port：22\n    username：root\n    password：QZDqzd65976787\n\n数据库：PostgreSQL\nport: 5432\n用户名：postgres\n密码：tongKen123\n\n'

In [2]:
# 数据库连接参数
conn =  psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
cur =  conn.cursor()

# val = [['1504479600', '2017/09/04 07:00:00', 4602.3, 4652, 4602.2, 4624.2, 5088813],
#        ['1504483200', '2017/09/04 08:00:00', 4624.2, 4630, 4564.3, 4589.2, 8407788],
#        ['1504486800', '2017/09/04 09:00:00', 4589.2, 4596.5, 4540.5, 4568.8, 6353670],
#        ['1504490400', '2017/09/04 10:00:00', 4568.8, 4578, 4551.4, 4571.1, 4545099]]
# 插入数据
# sql = """INSERT INTO xbtusd (timestamp , time , open , high, low, close , volume)  VALUES (\'{}\',\'{}\',{},{},{},{},{});"""

# for v in val:
#     sqlg =  = sql.format(v[0(v[0], v[1], v[2], v[3], v[4], v[5], v[6])
#     print(sqlg)
#        cur.execute(sql(sqlg)

sqllasttime = """ SELECT time from xbtusd_1m order by id limit 100"""

cur.execute(sqllasttime)
rows = cur.fetchall() # all rows in table
ltime = rows[0][0]
print(rows[0][0])


conn.commit()
cur.close()
conn.close()

2017-01-01 08:00:00


In [3]:
# Step 1：Load csv file and preprocess: T/5T/15T/30T/H/2H/4H/D/W/M
def preprocess(data_location, file):
    df = pd.read_csv(data_location + file)
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    df.index = pd.DatetimeIndex(df['time'])
    return df

# Step 2：转变数据周期：
def resample(df, period):
    convrted_df = df.resample(period).last()
    convrted_df['open'] = df['open'].resample(period).first()
    convrted_df['high'] = df['high'].resample(period).max()
    convrted_df['low'] = df['low'].resample(period).min()
    convrted_df['close'] = df['close'].resample(period).last()
    convrted_df['volume'] = df['volume'].resample(period).sum()
    # Keep rows with at least 5 non-NaN values
    convrted_df.dropna(thresh=5, inplace=True)
    convrted_df.index = convrted_df['time']
    convrted_df['time'] = pd.DatetimeIndex(convrted_df['time'])
    return convrted_df

# 第三步：为能够plot蜡烛图做准备：
def cnvrt_date(convrted_df):
    cnvrted_date_df = convrted_df.copy()
    cnvrted_date_df['date'] = mpd.date2num(cnvrted_date_df['time'].dt.to_pydatetime())
    return cnvrted_date_df

# 第四步：plot蜡烛图：
# Improved version:
def plot_candlestick(cnvrted_date_df):
    py_offline.init_notebook_mode()
    candle_df = go.Candlestick( x   = cnvrted_date_df.index,
                                open = cnvrted_date_df['open'],
                                high = cnvrted_date_df['high'],
                                low  = cnvrted_date_df['low'],
                                close= cnvrted_date_df['close'])
    data = [candle_df]
#     plot_url = py.plot(data, filename='extend plot', fileopt='extend')
    py_offline.iplot(data, filename='Candle Stick')
#     plt.gcf().clear()

In [ ]:
while True:
    # Period的频率不能低于SQL里的数据频率。
    period = '5T'
    conn = psycopg2.connect(database="bitmexdata", user="postgres", password="tongKen123", host="128.199.97.202", port="5432")
    sqllasttime = """ SELECT time, open, high, low, close, volume from xbtusd_1m order by id desc limit 200"""
    df = pd.read_sql(sqllasttime, con=conn)
    conn.close()
    
    df.columns = ['time', 'open', 'high', 'low', 'close', 'volume']
    df.index = pd.DatetimeIndex(df['time'])
    res = resample(df, period)
    cnvrted_date_df = cnvrt_date(res)

    plot_candlestick(cnvrted_date_df)
    time.sleep(10)
    clear_output()

## 改进实时绘图

In [16]:
# import plotly
# plotly.tools.set_credentials_file(username='bradsun91', api_key='9S0DCEFijrvEeEj9tw7m')

In [13]:
# import plotly.offline as pyo
# from plotly import tools
# import time

# pyo.init_notebook_mode(connected=True)

# trace = go.Scatter(
#     x=[1, 2, 3],
#     y=[4, 5, 6]
# )

# fig = tools.make_subplots(rows=1, cols=1)
# fig['data'].append(trace)
# pyo.iplot(fig)

# for i in range(10):
#     time.sleep(1)
#     trace = go.Scatter(
#         x=[i, i+1, i+2],
#         y=[4, 5, 6]
#     )
#     fig['data'].update(trace)
#     pyo.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



KeyboardInterrupt: 

In [15]:
# from dash.dependencies import Output, Event
# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# import numpy as np
# import pandas as pd
# import plotly
# import plotly.graph_objs as go

# dados = np.array([[1509608700000,0.00002246,0.00002246,0.00002246,0.00002246,100.00000000,1509609599999],
# [1509609600000,0.00002800,0.00002802,0.00002800,0.00002800,6832.00000000,1509610499999],
# [1509610500000,0.00002700,0.00002700,0.00002501,0.00002501,3936.00000000,1509611399999],
# [1509611400000,0.00002588,0.00002678,0.00002588,0.00002614,7125.00000000,1509612299999],
# [1509612300000,0.00002615,0.00002621,0.00002614,0.00002617,19318.00000000,1509613199999],
# [1509613200000,0.00002627,0.00002643,0.00002625,0.00002627,109218.00000000,1509614099999],
# [1509614100000,0.00002627,0.00002642,0.00002603,0.00002639,134825.00000000,1509614999999],
# [1509615000000,0.00002639,0.00002655,0.00002616,0.00002618,74432.00000000,1509615899999]
# ])

# columns = ['open_time', 'open', 'high', 'low','close', 'volume', 'close_time']

# df = pd.DataFrame(data=dados, columns=columns)

# last_id = 0

# app = dash.Dash(__name__)

# app.layout = html.Div(
#     html.Div(className='container-fluid',children=
# [
# 	html.Div(className='row', children=html.Div(dcc.Graph(id='live-graph', animate=True), className='col s12 m12 l12')),
# 	dcc.Interval(
# 		id='graph-update',
# 		interval=5000
# 	)
# ]),
# )

# @app.callback(
# Output('live-graph','figure'),
# events = [Event('graph-update','interval')]
# )
# def graph_update():
#     global df

#     ndf = df.iloc[0:2]
#     print(ndf)

#     last_id = ndf.iloc[0]['open_time']

#     ndf['data'] = pd.to_datetime(ndf['open_time'], unit='ms')
#     ndf.set_index('open_time', inplace=True)
#     ndf.round({'close': 8, 'open': 8, 'high': 8, 'low': 8})

#     data = [ dict(
#         type = 'candlestick',
#         open = ndf.open,
#         high = ndf.high,
#         low = ndf.low,
#         close = ndf.close,
#         x = ndf.data,
#         yaxis = 'y2',
#         name = 'Ripple',
#     )]

#     df = df.iloc[2:]
#     print(df)

#     return {'data': data, 'layout': {'title': str(last_id)}}

# external_css = ["https://cdnjs.cloudflare.com/ajax/libs/materialize/0.100.2/css/materialize.min.css"]
# for css in external_css:
#     app.css.append_css({"external_url": css})

# external_js = ['https://cdnjs.cloudflare.com/ajax/libs/materialize/0.100.2/js/materialize.min.js']
# for js in external_js:
#     app.scripts.append_script({'external_url': js})

# server = app.server
# dev_server = app.run_server(debug=True)

In [14]:
# import plotly.plotly as py
# import plotly.graph_objs as go

# data = [
#     go.Scatter(
#         x=[1, 2],
#         y=[3, 4]
#     )
# ]

# plot_url = py.plot(data, filename='my plot')

In [13]:
# import plotly.plotly as py
# import plotly.graph_objs as go

# trace0 = go.Scatter(
#     x=[1, 2],
#     y=[1, 2]
# )

# trace1 = go.Scatter(
#     x=[1, 2],
#     y=[2, 3]
# )

# trace2 = go.Scatter(
#     x=[1, 2],
#     y=[3, 4]
# )

# data = [trace0, trace1, trace2]

# # Take 1: if there is no data in the plot, 'extend' will create new traces.
# plot_url = py.plot(data, filename='extend plot', fileopt='extend')

In [12]:
# trace0 = go.Scatter(
#     x=[3, 4],
#     y=[2, 1]
# )

# trace1 = go.Scatter(
#     x=[3, 4],
#     y=[3, 2]
# )

# trace2 = go.Scatter(
#     x=[3, 4],
#     y=[4, 3]
# )

# data = [trace0, trace1, trace2]

# # Take 2: extend the traces on the plot with the data in the order supplied.
# plot_url = py.plot(data, filename='extend plot', fileopt='extend')